In [ ]:
#@markdown ## 구글드라이브 연동
#@markdown **생성한 이미지를 구글드라이브에 자동저장하려면 체크해주세요**
Google_Drive = False #@param {type:"boolean"}

#@markdown **초기모델(checkpoint)을 구글드라이브에 저장하려면 체크해주세요**
#@markdown <div><font color="red">모델 파일사이즈(2GB ~ 7GB)가 크니 구글드라이브 용량을 확인해주세요</div>
#@markdown <div><font color="red">대신 한번받으면 다음 실행부터는 빨라져요</div>
Checkpoint_Google_Save = False #@param {type:"boolean"}

#@markdown **ComfyUI Impact Pack**
ComfyUI_Impact_Pack = False  #@param {type:"boolean"}

#@markdown -----
#@markdown *아래는 설정할 필요 없어요*

#@markdown **구글드라이브에 연결 폴더**
Google_Drive_Dir = 'webui' #@param {type:"string"}

#@markdown **폴더 설명** : 미리 만드셔도 되고 없으면 자동으로 생성해요
#@markdown * webui/output : 생성된 이미지들이 저장되요
#@markdown * webui/checkpoint : 모델(checkpoint)를 넣어주면 읽어올수 있어요
#@markdown * webui/lora : 로라(LoRA)를 넣어주면 읽어올수 있어요
#@markdown * webui/lycoris : 라이코리스(LyCORIS)를 넣어주면 읽어올수 있어요
#@markdown * webui/embedding : 임베딩(Textual Inversion)를 넣어주면 읽어올수 있어요
#@markdown * webui/hyperwork : 하이퍼워크(Hyperworks)를 넣어주면 읽어올수 있어요
#@markdown * webui/wildcards : 와일드카드(Wildcards)를 넣어주면 읽어올수 있어요

#@markdown **초기모델(checkpoint) URL**
Checkpoint_Url = 'https://civitai.com/api/download/models/131579?type=Model&format=SafeTensor&size=full&fp=fp16' #@param {type:"string"}
#@markdown **초기모델 파일명**
Checkpoint_Filename = 'rundiffusion_xl.safetensors' #@param {type:"string"}
Workspace = 'ui'

if Google_Drive:
  from google.colab import drive
  drive.mount('/content/drive')

Checkpoint_SavePath = f'/content/{Workspace}/models/checkpoints'
if Google_Drive and Checkpoint_Google_Save:
  Checkpoint_SavePath = f'/content/{Workspace}/models/checkpoints/google'

!apt -y install -qq aria2

%cd /content
!wget https://raw.githubusercontent.com/neuralninja22/colab/master/misc/install_comfyui_sdxl.sh -O /content/install.sh
!bash /content/install.sh {Workspace} {ComfyUI_Impact_Pack}

%cd /content/{Workspace}

# 구글드라이브 연결
!wget https://raw.githubusercontent.com/neuralninja22/colab/master/misc/link_google_drive_comfyui.sh -O /content/link_google_drive_comfyui.sh
!bash /content/link_google_drive_comfyui.sh {Workspace} {Google_Drive} {Google_Drive_Dir}

# checkpoint
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{Checkpoint_Url}" -d {Checkpoint_SavePath} -o {Checkpoint_Filename}

# upscaler
!wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x4.pth -P /content/{Workspace}/models/upscale_models/
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/{Workspace}/models/upscale_models -o 4x-UltraSharp.pth

# vae
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d /content/{Workspace}/models/vae -o vae-ft-mse-840000-ema-pruned.vae.pt

# Run ComfyUI with localtunnel (Recommended Way)
!npm install -g localtunnel

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server